## Yapı Kredi QnA BoT version 0.1

In [1]:
#!!!!There are duplicate Quetions and Answers 
#It does not effect te result. It reponses the best match

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import requests

In [2]:
def removeCharacter(list_of_sent , char):
    new_list = []
    for each in list_of_sent:
        to_be_added = each.replace(char, "")
        new_list.append(to_be_added)
    return new_list

In [3]:
questions = []
answers = []

f_q = open("/Users/cankutcoskun/Desktop/ChatBotProject/QnA/questions.txt")
f_a = open("/Users/cankutcoskun/Desktop/ChatBotProject/QnA/answers.txt")

questions = removeCharacter(f_q.readlines(), "\n")
answers = removeCharacter(f_a.readlines(), "\n")


f_q.close()
f_a.close()

In [4]:
def processDocument(path):
    
    PARAMS = {'rawDocPathName': path}
    response = requests.get("http://localhost:8080/nlpPipeLine/document", params = PARAMS)
    
    if response.status_code == requests.codes.ok:
        resp_text = response.text
        resp_list = resp_text.rstrip("]").lstrip("[").replace('"','').split("],[")

        all_tokens = []

        for r in resp_list:
            tokens = r.split(",")
            all_tokens.append(tokens)
        
        processed_doc = []

        for tks in all_tokens:
            temp = " ".join(tks)
            processed_doc.append(temp)
        
        return processed_doc

    else:
        raise Exception("WARNING REQUEST DOES NOT WORK PROPERLY \n response code: " + str(response.status_code))

In [5]:
processed_questions = processDocument("/Users/cankutcoskun/Desktop/ChatBotProject/QnA/questions.txt")
processed_answers = processDocument("/Users/cankutcoskun/Desktop/ChatBotProject/QnA/answers.txt")

##Edge
processed_answers[145] = ""

In [6]:
processed_qnas = []

for i in range(len(processed_questions)):
    qna = processed_questions[i] + " "+processed_answers[i]
    processed_qnas.append(qna)

In [7]:
processed_qnas

['fatura öde işlem ücret al kredi kart fatura öde alın işlem ücret ürün hizmet ücret sayfa yer al kredi kart işlem ücret anlaş kurum fatura sgk prim öde başlık alt incele ayrıca işlem onay adım al ücret tutar gör',
 'mobil şube banka işlem yap sms çağrı al mobil şube banka işlem yap telefon sms çağrı gel uygula arka plan düş kısa süre çalış devam et belirle süre içeri uygula tekrar dön işlem devam et belirle süre uygula dön tekrar gir yap işlem yap',
 'world üye işyeri world özellik kazan kredi kart başka banka pos geçir world puan ver mi world üye işyeri world özellik kazan kredi kart yapı kredi pos geçir başka banka pos kullan hesap yapı kredi worldcard ver worldpuan puan kesinti ol',
 'vade fark sat iste talep bulun vade fark sat yetki aç yapı kredi şube başvuru yap',
 'kredi faiz oran düşür mi kredi faiz bedel kısım baş öde kredi faiz oran in saye aylık taksit önem ölçü azal mevcut birikim şekil değer faiz oran ilan et sabit faiz alt kredi vade boyunca az tutar taksit öde yap',
 'e

In [13]:
vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=1.0, min_df = 0, use_idf = True)
bow = vectorizer.fit_transform(processed_qnas)

freqs = [(word, bow.getcol(idx).sum()) for word, idx in vectorizer.vocabulary_.items()]
results = sorted (freqs, key = lambda x: -x[1])
print("Number of unique words (size of dictionary):", len(vectorizer.vocabulary_.items()))

Number of unique words (size of dictionary): 1108


In [17]:
feature_names = vectorizer.get_feature_names()
corpus_index = [n for n in processed_questions]

df = pd.DataFrame(bow.todense(), columns=feature_names)

#( num of questions , size of vocab)
print(df.shape)

df.head(5)

(393, 1108)


,abone,acil,ad,adana,adet,adios,adres,adsl,adım,aile,...,şahıs,şart,şekil,şifre,şimdi,şirket,şoför,şu,şube,şüphe
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.176619,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.112766,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.125952,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.165547,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [26]:
def processInput(raw_inp):
    
    emptyList = []
    
    PARAMS = {'rawInput':raw_inp} 

    response = requests.get("http://localhost:8080/nlpPipeLine/SingleInput", params = PARAMS)
    
    print("Response Code: ", response.status_code)

    if response.status_code == requests.codes.ok:
        text = response.text
        return True, text.rstrip("]").lstrip("[").replace('"','').split(",")
    else:
        return False, emptyList
        

In [27]:
def rankingTable(scores):
    ranking_table = []

    for i in range(len(scores)):
        temp = (i,scores[i])
        ranking_table.append(temp)
        
    return ranking_table

In [28]:
def bestMatchIdx(raw_inp):

    bestMatch_idx = -1
    
    check , inp_tokens = processInput(raw_inp) 
    
    print(inp_tokens)
    
    emptyList = []
    
    if inp_tokens == emptyList:
        return -1
    
    processed_inp = ""
    
    if check:
        processed_inp = " ".join(inp_tokens)
    else:
        return False
    
    processed_questions.append(processed_inp)
    tfidf = vectorizer.fit_transform(processed_questions)
    vals = cosine_similarity(tfidf[-1], tfidf)
    
    processed_questions.pop()
    
    best_match_idx = vals.argsort()[0][-2]
    
    score_list = vals.tolist()[0]
    
    ranking_table = rankingTable(score_list[0:-1])
    
    #for each in ranking_table:
    #   print(each)
        
    return best_match_idx

In [29]:
def userInputBestMatch(inp):
    bestIdx = bestMatchIdx(inp)
    print(bestIdx)
    
    if bestIdx == -1:
        print ("Üzgünüm sizi anlayamadım\n")
    else:
        print(questions[bestIdx])
        print(answers[bestIdx])

    

In [ ]:
print("sonlandırmak için lütfen 'çıkış' yazın\n")

inp = input("Ne öğrenmek istersin? \n\n") 

while inp != "çıkış":
    
    userInputBestMatch(inp)
    inp = input("Ne öğrenmek istersin?  \n\n")
                
                

sonlandırmak için lütfen 'çıkış' yazın

Ne öğrenmek istersin? 

hesap kendiliğinden yenilenir mi?
Response Code:  200
['hesap', 'kendiliğinden', 'yenile']
350
Kredi kartının yenilenmesini nasıl yapabiliriz?
Kartınızın yenilenmesi için 444 0 444 / 0850 222 0 444 Yapı Kredi Telefon Bankacılığı'nı arayarak müşteri temsilcilerimizden destek alabilirsiniz.
